## Install and Import

In [1]:
%pip install -q rdkit pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from rdkit import Chem, RDLogger
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage
import numpy as np
import tensorflow as tf
from tensorflow import keras

RDLogger.DisableLog("rdApp.*")

## Dataset

In [3]:
class DataHelper():
    def __init__(self):
        csv_path = tf.keras.utils.get_file("qm9.csv", "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/qm9.csv")
        self.data = list()
        with open(csv_path, "r")  as fin:
            for line in fin.readlines()[1:]:
                self.data.append(line.split(",")[1])

    def __getitem__(self, idx: int):
        smiles = self.data[idx]
        print(f"SMILES: {smiles}")
        mol = Chem.MolFromSmiles(smiles)
        print(f"Number of Heavy Atoms: {mol.GetNumHeavyAtoms()}")
        return mol

In [4]:
DH = DataHelper()

29856825/29856825 [==============================] - 0s 0us/step


## Utilities

In [5]:
atom_mapping = {
    "C": 0,
    0: "C",
    "N": 1,
    1: "N",
    "O": 2,
    2: "O",
    "F": 3,
    3: "F",
}

bond_mapping = {
    "SINGLE": 0,
    0: Chem.BondType.SINGLE,
    "DOUBLE": 1,
    1: Chem.BondType.DOUBLE,
    "TRIPLE": 2,
    2: Chem.BondType.TRIPLE,
    "AROMATIC": 3,
    3: Chem.BondType.AROMATIC,
}

NUM_ATOMS = 9  # Maximum number of atoms
ATOM_DIM = 4 + 1  # Number of atom types
BOND_DIM = 4 + 1  # Number of bond types
LATENT_DIM = 64  # Size of the latent space


def smiles_to_graph(smiles):
    # Converts SMILES to molecule object
    molecule = Chem.MolFromSmiles(smiles)

    # Initialize adjacency and feature tensor
    adjacency = np.zeros((BOND_DIM, NUM_ATOMS, NUM_ATOMS), "float32")
    features = np.zeros((NUM_ATOMS, ATOM_DIM), "float32")

    # loop over each atom in molecule
    for atom in molecule.GetAtoms():
        i = atom.GetIdx()
        atom_type = atom_mapping[atom.GetSymbol()]
        features[i] = np.eye(ATOM_DIM)[atom_type]
        # loop over one-hop neighbors
        for neighbor in atom.GetNeighbors():
            j = neighbor.GetIdx()
            bond = molecule.GetBondBetweenAtoms(i, j)
            bond_type_idx = bond_mapping[bond.GetBondType().name]
            adjacency[bond_type_idx, [i, j], [j, i]] = 1

    # Where no bond, add 1 to last channel (indicating "non-bond")
    # Notice: channels-first
    adjacency[-1, np.sum(adjacency, axis=0) == 0] = 1

    # Where no atom, add 1 to last column (indicating "non-atom")
    features[np.where(np.sum(features, axis=1) == 0)[0], -1] = 1

    return adjacency, features


def graph_to_molecule(graph):
    # Unpack graph
    adjacency, features = graph

    # RWMol is a molecule object intended to be edited
    molecule = Chem.RWMol()

    # Remove "no atoms" & atoms with no bonds
    keep_idx = np.where(
        (np.argmax(features, axis=1) != ATOM_DIM - 1)
        & (np.sum(adjacency[:-1], axis=(0, 1)) != 0)
    )[0]
    features = features[keep_idx]
    adjacency = adjacency[:, keep_idx, :][:, :, keep_idx]

    # Add atoms to molecule
    for atom_type_idx in np.argmax(features, axis=1):
        atom = Chem.Atom(atom_mapping[atom_type_idx])
        _ = molecule.AddAtom(atom)

    # Add bonds between atoms in molecule; based on the upper triangles
    # of the [symmetric] adjacency tensor
    (bonds_ij, atoms_i, atoms_j) = np.where(np.triu(adjacency) == 1)
    for (bond_ij, atom_i, atom_j) in zip(bonds_ij, atoms_i, atoms_j):
        if atom_i == atom_j or bond_ij == BOND_DIM - 1:
            continue
        bond_type = bond_mapping[bond_ij]
        molecule.AddBond(int(atom_i), int(atom_j), bond_type)

    # Sanitize the molecule; for more information on sanitization, see
    # https://www.rdkit.org/docs/RDKit_Book.html#molecular-sanitization
    flag = Chem.SanitizeMol(molecule, catchErrors=True)
    # Let's be strict. If sanitization fails, return None
    if flag != Chem.SanitizeFlags.SANITIZE_NONE:
        return None

    return molecule

## Generate Training Set

In [6]:
adjacency_tensor, feature_tensor = [], []
for smiles in DH.data[::10]:
    adjacency, features = smiles_to_graph(smiles)
    adjacency_tensor.append(adjacency)
    feature_tensor.append(features)

adjacency_tensor = np.array(adjacency_tensor)
feature_tensor = np.array(feature_tensor)

print("adjacency_tensor.shape =", adjacency_tensor.shape)
print("feature_tensor.shape =", feature_tensor.shape)

adjacency_tensor.shape = (13389, 5, 9, 9)
feature_tensor.shape = (13389, 9, 5)


## Quantum Graph Generator - TODO

### Quantum LSTM Cell

In [ ]:
%pip install pennylane

In [13]:
import pennylane as qml

In [14]:
class QLSTMCell(keras.Model):
    def __init__(self, input_size, hidden_size, n_qubits, n_layers=1, backend="default.qubit"):
        super(QLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.backend = backend

        self.wires_forget = [f"wire_forget_{i}" for i in range(self.n_qubits)]
        self.wires_inputs = [f"wire_inputs_{i}" for i in range(self.n_qubits)]
        self.wires_update = [f"wire_update_{i}" for i in range(self.n_qubits)]
        self.wires_output = [f"wire_output_{i}" for i in range(self.n_qubits)]

        self.dev_forget = qml.device(self.backend, wires=self.wires_forget)
        self.dev_inputs = qml.device(self.backend, wires=self.wires_inputs)
        self.dev_update = qml.device(self.backend, wires=self.wires_update)
        self.dev_output = qml.device(self.backend, wires=self.wires_output)

        def _circuit_forget(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires_forget)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires_forget)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires_forget]
        self.qlayer_forget = qml.QNode(_circuit_forget, self.dev_forget, interface="tf")

        def _circuit_input(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires_inputs)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires_inputs, rotation=qml.RY)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires_inputs]
        self.qlayer_inputs = qml.QNode(_circuit_input, self.dev_inputs, interface="tf")

        def _circuit_update(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires_update)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires_update)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires_update]
        self.qlayer_update = qml.QNode(_circuit_update, self.dev_update, interface="tf")

        def _circuit_output(inputs, weights):
            qml.templates.AngleEmbedding(inputs, wires=self.wires_output)
            qml.templates.BasicEntanglerLayers(weights, wires=self.wires_output)
            return [qml.expval(qml.PauliZ(wires=w)) for w in self.wires_output]
        self.qlayer_output = qml.QNode(_circuit_output, self.dev_output, interface="tf")

        weight_shapes = {"weights": (n_layers, n_qubits)}

        self.cell = keras.layers.Dense(input_size + hidden_size, use_bias=True)
        # default args = xavier_uniform for weight and zeros for bias

        self.VQC = {
            'forget': qml.qnn.KerasLayer(self.qlayer_forget, weight_shapes, n_qubits),
            'inputs': qml.qnn.KerasLayer(self.qlayer_inputs, weight_shapes, n_qubits),
            'update': qml.qnn.KerasLayer(self.qlayer_update, weight_shapes, n_qubits),
            'output': qml.qnn.KerasLayer(self.qlayer_output, weight_shapes, n_qubits)
        }

        self.clayer_out = keras.layers.Dense(n_qubits, use_bias=False)

    def call(self, x, hidden):
        hx, cx = hidden
        gates = tf.concat([x, hx], axis=1)
        gates = self.cell(gates)

        for layer in range(self.n_layers):
            ingate = keras.activations.sigmoid(self.clayer_out(self.VQC['forget'](gates)))
            forgetgate = keras.activations.sigmoid(self.clayer_out(self.VQC['inputs'](gates)))
            cellgate = keras.activations.tanh(self.clayer_out(self.VQC['update'](gates)))
            outgate = keras.activations.sigmoid(self.clayer_out(self.VQC['forget'](gates)))

            cy = tf.math.multiply(cx, forgetgate) + tf.math.multiply(ingate, cellgate)
            hy = tf.math.multiply(outgate, keras.activations.tanh(cy))

        return (hy, cy)


In [15]:
QL = QLSTMCell(8, 8, 16)

In [16]:
QL([[152], [191]], [[[14003], [3356]], [[16025], [3332]]])

(<tf.Tensor: shape=(2, 16), dtype=float32, numpy=
 array([[0.49785537, 0.5111109 , 0.50267434, 0.48306435, 0.4997821 ,
         0.54256725, 0.53681946, 0.47413915, 0.50309986, 0.51466304,
         0.4859694 , 0.4678425 , 0.53466487, 0.49068442, 0.50893986,
         0.49183512],
        [0.4705162 , 0.5169401 , 0.54420304, 0.4806176 , 0.5392125 ,
         0.45226473, 0.4879713 , 0.53293705, 0.50252223, 0.54763067,
         0.5468684 , 0.5714903 , 0.45921072, 0.49655184, 0.4488556 ,
         0.5574867 ]], dtype=float32)>,
 <tf.Tensor: shape=(2, 16), dtype=float32, numpy=
 array([[7871.2744, 8014.5356, 8092.75  , 7866.7485, 8050.4917, 8327.298 ,
         8325.258 , 7784.384 , 7990.8037, 8135.0366, 7840.888 , 7825.442 ,
         8283.218 , 7905.5854, 8006.092 , 7930.255 ],
        [1531.287 , 1722.0348, 1786.6824, 1629.1345, 1794.9073, 1543.3058,
         1656.9673, 1772.2563, 1703.1938, 1784.28  , 1792.6654, 1930.8656,
         1504.2983, 1646.7672, 1484.5356, 1845.7428]], dtype=float32)>

In [17]:
QL.summary()

Model: "qlstm_cell"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             multiple                  48        
                                                                 
 keras_layer (KerasLayer)    multiple                  16        
                                                                 
 keras_layer_1 (KerasLayer)  multiple                  16        
                                                                 
 keras_layer_3 (KerasLayer)  multiple                  16        
                                                                 
 keras_layer_2 (KerasLayer)  multiple                  16        
                                                                 
 dense_4 (Dense)             multiple                  256       
                                                                 
Total params: 368
Trainable params: 368
Non-trainable pa

### Actual Generator - VERY BUGGY

In [65]:
class GraphGenerator(keras.Model):
    def __init__(self, H_inputs, H, z_dim, N, rw_len, temp):
        '''
            H_inputs: input dimension
            H:        hidden dimension
            z_dim:    latent dimension
            N:        number of nodes (needed for the up and down projection)
            rw_len:   number of LSTM cells
            temp:     temperature for the gumbel softmax
        '''
        super(GraphGenerator, self).__init__()
        self.intermediate = keras.layers.Dense(H)
        self.c_up = keras.layers.Dense(H)
        self.h_up = keras.layers.Dense(H)
        self.qlstm = QLSTMCell(input_size=H_inputs, hidden_size=H, n_qubits=N)
        self.W_up = keras.layers.Dense(N)
        self.W_down = keras.layers.Dense(H_inputs, use_bias=False)
        self.rw_len = rw_len
        self.temp = temp
        self.H, self.N = H, N
        self.H_inputs = H_inputs
        self.latent_dim = z_dim

    def sample_latent(self, num_samples):
        return tf.random.normal([num_samples, self.latent_dim])

    # I HAVE NO IDEA HOW TO DO THIS IN TENSORFLOW
    def init_hidden(self, batch_sz):
        weight = next(self.parameters).data
        return weight.new(batch_sz, self.H_inputs).zero_()
    # HELP WITH FUNCTION ABOVE

    def sample_gumbel(self, logits, eps=1e-20):
        U = tf.random.normal([logits.shape])
        return -tf.math.log(-tf.math.log(U + eps) + eps)

    def gumbel_softmax(self, logits, temp):
        gumbel = self.sample_gumbel(logits)
        y = logits + gumbel
        y = tf.nn.softmax(y / temp, axis=1)
        return y

    def call(self, latent, inputs):
        intermediate = keras.activations.tanh(self.intermediate(latent))
        hc = (
            keras.activations.tanh(self.h_up(intermediate)),
            keras.activations.tanh(self.c_up(intermediate))
        )
        out = []
        for i in range(self.rw_len):
            hh, cc = self.qlstm(inputs, hc)
            hc = (hh, cc)
            h_up = self.W_up(hh)
            h_sample = self.gumbel_softmax(h_up, self.temp)
            inputs = self.W_down(h_sample)
            out.append(h_sample)
        return tf.stack(out, axis=1)

    def sample_reg(self, num_samples):
        noise = self.sample_latent(num_samples)
        inp_zeroes = self.init_hidden(num_samples)
        gen_data = self(noise, inp_zeroes)
        return gen_data

    #Not sure if this works
    def sample_disc(self, num_samples):
        proba = tf.stop_gradient(self.sample_reg(num_samples))
        return np.argmax(proba.numpy(), axis=2)


In [67]:
QG = GraphGenerator(16, 16, 16, 16, 1, 0.5)

In [68]:
QG([314.1592653], [7438, 9465])

ValueError: ignored

## Graph Discriminator

### Relational Graph Convolution Layer

In [7]:
class RelationalGraphConvLayer(keras.layers.Layer):
    def __init__(
        self,
        units=128,
        activation="relu",
        use_bias=False,
        kernel_initializer="glorot_uniform",
        bias_initializer="zeros",
        kernel_regularizer=None,
        bias_regularizer=None,
        **kwargs
    ):
        super().__init__(**kwargs)

        self.units = units
        self.activation = keras.activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = keras.initializers.get(kernel_initializer)
        self.bias_initializer = keras.initializers.get(bias_initializer)
        self.kernel_regularizer = keras.regularizers.get(kernel_regularizer)
        self.bias_regularizer = keras.regularizers.get(bias_regularizer)

    def build(self, input_shape):
        bond_dim = input_shape[0][1]
        atom_dim = input_shape[1][2]

        self.kernel = self.add_weight(
            shape=(bond_dim, atom_dim, self.units),
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            trainable=True,
            name="W",
            dtype=tf.float32,
        )

        if self.use_bias:
            self.bias = self.add_weight(
                shape=(bond_dim, 1, self.units),
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                trainable=True,
                name="b",
                dtype=tf.float32,
            )

        self.built = True

    def call(self, inputs, training=False):
        adjacency, features = inputs
        # Aggregate information from neighbors
        x = tf.matmul(adjacency, features[:, None, :, :])
        # Apply linear transformation
        x = tf.matmul(x, self.kernel)
        if self.use_bias:
            x += self.bias
        # Reduce bond types dim
        x_reduced = tf.reduce_sum(x, axis=1)
        # Apply non-linear transformation
        return self.activation(x_reduced)

### Actual Disciminator

In [8]:
def GraphDiscriminator(
    gconv_units, dense_units, dropout_rate, adjacency_shape, feature_shape
):

    adjacency = keras.layers.Input(shape=adjacency_shape)
    features = keras.layers.Input(shape=feature_shape)

    # Propagate through one or more graph convolutional layers
    features_transformed = features
    for units in gconv_units:
        features_transformed = RelationalGraphConvLayer(units)(
            [adjacency, features_transformed]
        )

    # Reduce 2-D representation of molecule to 1-D
    x = keras.layers.GlobalAveragePooling1D()(features_transformed)

    # Propagate through one or more densely connected layers
    for units in dense_units:
        x = keras.layers.Dense(units, activation="relu")(x)
        x = keras.layers.Dropout(dropout_rate)(x)

    # For each molecule, output a single scalar value expressing the
    # "realness" of the inputted molecule
    x_out = keras.layers.Dense(1, dtype="float32")(x)

    return keras.Model(inputs=[adjacency, features], outputs=x_out)

In [9]:
NUM_ATOMS = 9  # Maximum number of atoms
ATOM_DIM = 4 + 1  # Number of atom types
BOND_DIM = 4 + 1  # Number of bond types
LATENT_DIM = 64  # Size of the latent space

In [10]:
discriminator = GraphDiscriminator(
    gconv_units=[128, 128, 128, 128],
    dense_units=[512, 512],
    dropout_rate=0.2,
    adjacency_shape=(BOND_DIM, NUM_ATOMS, NUM_ATOMS),
    feature_shape=(NUM_ATOMS, ATOM_DIM),
)

In [11]:
discriminator.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 9, 9)]    0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 9, 5)]       0           []                               
                                                                                                  
 relational_graph_conv_layer (R  (None, 9, 128)      3200        ['input_1[0][0]',                
 elationalGraphConvLayer)                                         'input_2[0][0]']                
                                                                                                  
 relational_graph_conv_layer_1   (None, 9, 128)      81920       ['input_1[0][0]',            

## Compile Final Model - TODO